# Módulo 5: Crear un agente con Agent Framework

En este ejercicio, utilizarás Microsoft Agent Framework para crear un agente de IA que procesa informes de gastos de viajes y operaciones aeroespaciales.

### Cargar librerías

In [ ]:
import os
import asyncio
from pathlib import Path
from agent_framework import AgentThread, ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field
from typing import Annotated

### Visualizar los datos de gastos

In [ ]:
script_dir = Path.cwd()
file_path = script_dir / 'data.txt'
with file_path.open('r') as file:
    expenses_data = file.read()
    print("Datos de gastos:")
    print(expenses_data)

### Definir una función de herramienta (Tool Function)

Creamos una función `send_email` que el agente podrá utilizar. Esta función simula el envío de un correo electrónico imprimiendo los detalles en la consola.

In [ ]:
def send_email(
    to: Annotated[str, Field(description="Email recipient")],
    subject: Annotated[str, Field(description="Email subject")],
    body: Annotated[str, Field(description="Email body")]):
        print("\n--- Simulated Email ---")
        print("To:", to)
        print("Subject:", subject)
        print("Body:")
        print(body)
        print("----------------------")

print("Function 'send_email' defined")

### Crear y ejecutar el agente

Ahora, creamos un `ChatAgent` con instrucciones específicas y le proporcionamos la función `send_email` como herramienta. Luego, le enviamos los datos de gastos junto con un prompt.

In [ ]:
async def process_expenses(prompt, data):
    print("Creating expenses agent...")
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="expenses_agent",
            instructions="""
            You are an AI assistant for company expense reports.
            When a user sends expense data and requests a claim, use the tool to send an email to finance@example.com
            with subject 'Expense Report - Aerospace Operations' and body containing:
            - Itemized expenses
            - Technical purpose
            - Total amount in EUR
            Then confirm to user that report has been sent.
            """,
            tools=send_email,
        ) as agent,
    ):
        try:
            print(f"Processing expense data for: {prompt}")
            prompt_messages = [f"{prompt}: {data}"]
            print("Running agent...")
            response = await agent.run(prompt_messages)
            print("\nAgent response:")
            print(f"{response}")
        except Exception as e:
            print(f"Error occurred: {e}")

# Execute processing
print("Starting expense processing...")
user_prompt = "Send aerospace operations expense report"
await process_expenses(user_prompt, expenses_data)
print("Expense processing completed")